In [ ]:
import requests
import os
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime, timedelta

try:
    os.makedirs('./tmp_data')
    print('crawling start')
except:
    print('crawling start')

start_date = datetime.strptime('2022-04-07', '%Y-%m-%d')
end_date = datetime.strptime('2023-03-22', '%Y-%m-%d')
delta = timedelta(days=1)

dates = []
current_date = start_date

while current_date <= end_date:
    dates.append(current_date.strftime('%Y-%m-%d'))
    current_date += delta


data_list = []
for date in dates:
    print(date)
    data_dict = {
        "기준시간": [],
        "공급능력": [],
        "현재부하": [],
        "공급예비력": [],
        "운영예비력": [],
        "운영예비율": [],
    }
    url = f'http://www.happydr.co.kr/?act=&mid=CurrentSupplyView&vid=&view=chart&option_date={date}'

    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')
    tbody = soup.find('tbody')

    trs = tbody.find_all('tr', {'y': ''})

    for tr in trs:
        row = [td.text for td in tr.find_all('td')]

        tmp = {
            column: crawl_data
            for column, crawl_data in zip(data_dict, row)
        }
        data_list.append(tmp)

    df = pd.DataFrame(data_list)
    df.to_csv(f'./tmp_data/real_sup_electro_{str(date)}.csv')
